Querying astronomical databases with Astroquery
========================================

Authors: Erik Tollerud & Brigitta Sipőcz

astroquery is an astropy coordinated package. The vision is to have a uniform pythonic interface
for data consumers, a de-facto gateway to data. Most modules are developed independently, some data providers
do support modules in astroquery, others are developed by volunteers.

astroquery will always remain independent from core astropy: it has to keep up with rapid changes on servers.

Developer versions are "Continuously deployed", these versions can be pip installed with 
`pip install --pre -U astroquery`. Generarilly it's always best to have the most recent developer version.

Modules heavily use astropy, and thus can work with astropy objects. (E.g. coordinates can be provided as `SkyCoord`s, results are usually returned as astropy `Tables`.

Homepage, including installation instructions:
    https://astroquery.readthedocs.io/
    
If you use astroquery, please cite the the paper:  [Ginsburg, Sipőcz, Brasseur et al, AJ, 157, 98](https://ui.adsabs.harvard.edu/abs/2019AJ....157...98G)

## Vizier, SDSS, and XMatch

Download a table from a paper.

Documentation and several examples: 
* https://astroquery.readthedocs.io/en/latest/vizier/vizier.html
* https://astroquery.readthedocs.io/en/latest/xmatch/xmatch.html

In [ ]:
# Import useful Astropy functionality
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:
from astroquery.vizier import Vizier

vizier = Vizier()
vizier.ROW_LIMIT = -1  # default is too small

# Download BD parallaxes from e.g. Dupuy&Liu 2012 ApJS

LT_parallax = vizier.get_catalogs(catalog="J/ApJS/201/19/plx-phot")[0]

In [ ]:
LT_parallax

Now focus on the first object in this list. First, all SDSS sources from a closeby region.

Hint: most astroquery modules have a `query_region()` method when it makes sense to query for coordinate. 

In [ ]:
# First create a SkyCoord object
position = SkyCoord(LT_parallax[0]['RAJ2000'], LT_parallax[0]['DEJ2000'], unit="deg")

# Get closeby sources from SDSS
from astroquery.sdss import SDSS
sdss = SDSS()
closeby_sources = sdss.query_region(position, radius=0.1 * u.deg, timeout=3600)

In [ ]:
closeby_sources

## Exercise: get the SDSS image of the region

Maybe also create a colour image, following the astropy docs http://docs.astropy.org/en/stable/visualization/rgb.html

Hint: ``dir(SDSS)`` will give you the list of all available methods. E.g. there is `get_images()`, `get_spectra()`, etc.

## Write SQL query and cross match is with other catalogues

Some modules support SQL queries (the server side need to support this, too). You can then use XMatch 
to cross match with other catalogues, e.g. with WISE

In [ ]:
from astroquery.sdss import SDSS
sdss = SDSS()
query = "select top 100 z, ra, dec, bestObjID from specObj where class = 'galaxy' and z > 0.3 and zWarning = 0"

results = sdss.query_sql(query)

In [ ]:
from astroquery.xmatch import XMatch
xmatch = XMatch()

SDSS_WISE_match = table= XMatch.query(cat1=results,
                                      cat2='vizier:ii/328/allwise',
                                      max_distance=3 * u.arcsec, colRA1='ra',colDec1='dec')

In [ ]:
SDSS_WISE_match

## Exercise: make a query that actually makes sense, create a plot.



## Exercise: Based on your science interest, choose an archive and make astroquery queries

Full list of modules available: https://astroquery.readthedocs.io/en/latest/

Some examples for the most commonly used ones:

 * astroquery.alma
 * astroquery.eso
 * astroquery.gaia
 * astroquery.jplhorizon
 * astroquery.mast
 * astroquery.sdss
 * astroquery.vizier
 
Verbosity of the narrative documentation varies, but methods have documentation in their docstrings.

## Resources elsewhere on astroquery exist, notebooks e.g.:

Notebooks on MAST: https://github.com/spacetelescope/notebooks
* [Cutout of the TESS FFIs using Astrocut and Astroquery](https://spacetelescope.github.io/notebooks/notebooks/MAST/TESS/interm_tesscut_astroquery/interm_tesscut_astroquery.html)
* [Intermediate: Search and Download GI Program Light Curves](https://spacetelescope.github.io/notebooks/notebooks/MAST/TESS/interm_gi_query/interm_gi_query.html)
* etc.

